# Behavioral Cloning



In [4]:
import csv
import cv2
import numpy

### Load data

In [6]:
lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
images = []
angles = []
for line in lines[1:]:
    sourcepath = line[0]
    filename = sourcepath.split('/')[-1]
    currentpath = 'data/IMG/' + filename
    image = cv2.imread(currentpath)
    images.append(image)
    angle = float(line[3])
    angles.append(angle)

X_train = numpy.array(images)
y_train = numpy.array(angles)

### Build Keras model

In [17]:
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint

In [21]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))  # Normalize and center data.
model.add(Conv2D(16, 3, 3, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(1))

model.summary()
model.compile(optimizer='adam', loss='mse')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 160, 320, 3)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 158, 318, 16)  448         lambda_2[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_7 (MaxPooling2D)    (None, 79, 159, 16)   0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 77, 157, 32)   4640        maxpooling2d_7[0][0]             
___________________________________________________________________________________________

### Train model

In [ ]:
checkpointer = ModelCheckpoint(filepath='weights/model1.hdf5', verbose=1, save_best_only=True)

model.fit(X_train, y_train, validation_split=0.2, shuffle=True, callbacks=[checkpointer], verbose=1)

### Load best weights and save model

In [20]:
model.load_weights('weights/model1.hdf5')
model.save('model1.h5')